In [3]:
# ▶︎ 1. Imports & Device
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ▶︎ 2. Data transforms & loaders
# CIFAR-10 images are 32×32; VGG expects ≥224×224
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

val_transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

train_ds = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=train_transform
)
val_ds = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=val_transform
)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False, num_workers=4)

# ▶︎ 3. Load & modify VGG16
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).to(device)

# Freeze convolutional backbone
for param in model.features.parameters():
    param.requires_grad = False

# Replace classifier head
n_classes = 10
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(4096, 1024),
    nn.ReLU(inplace=True),
    nn.Dropout(0.5),
    nn.Linear(1024, n_classes),
).to(device)

# ▶︎ 4. Loss & optimizer (only head params)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)


# ▶︎ 5. Training / validation functions
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)
    return running_loss / total, correct / total


def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * imgs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
    return running_loss / total, correct / total


# ▶︎ 6. Full training loop
n_epochs = 10
for epoch in range(1, n_epochs + 1):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = eval_epoch(model, val_loader, criterion)
    print(
        f"Epoch {epoch}/{n_epochs}  "
        f"Train: loss={train_loss:.4f}, acc={train_acc:.4f}  "
        f"Val:   loss={val_loss:.4f}, acc={val_acc:.4f}"
    )

print("✅ Fine-tuning complete!")


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch 1/10  Train: loss=1.0300, acc=0.6302  Val:   loss=0.4779, acc=0.8305
Epoch 2/10  Train: loss=0.8603, acc=0.6948  Val:   loss=0.4396, acc=0.8458
Epoch 3/10  Train: loss=0.8142, acc=0.7126  Val:   loss=0.3994, acc=0.8615
Epoch 4/10  Train: loss=0.7776, acc=0.7254  Val:   loss=0.3789, acc=0.8711
Epoch 5/10  Train: loss=0.7645, acc=0.7296  Val:   loss=0.3824, acc=0.8680
Epoch 6/10  Train: loss=0.7406, acc=0.7390  Val:   loss=0.3711, acc=0.8750
Epoch 7/10  Train: loss=0.7186, acc=0.7451  Val:   loss=0.3745, acc=0.8726
Epoch 8/10  Train: loss=0.7071, acc=0.7495  Val:   loss=0.3608, acc=0.8773
Epoch 9/10  Train: loss=0.6956, acc=0.7544  Val:   loss=0.3520, acc=0.8818
Epoch 10/10  Train: loss=0.6837, acc=0.7592  Val:   loss=0.3466, acc=0.8839
✅ Fine-tuning complete!
